# Pulizia "movies.csv"

In [1]:
import pandas as pd
import os

In [2]:
# Risali alla cartella "Solutions"
script_dir = os.getcwd()
solutions_dir = os.path.abspath(os.path.join(script_dir, ".."))

# Imposta "Data" come directory di lavoro
data_dir = os.path.join(solutions_dir, "Data")
os.chdir(data_dir)  # Cambia la directory di lavoro

# Ora puoi riferirti direttamente ai file dentro "Data"
file_path = os.path.join("main_data", "movies.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(data_dir, "clean_data")

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [4]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

# Converti "data" e "anno" in numerico (float se ci sono NaN)
df["minute"] = pd.to_numeric(df["minute"], errors="coerce")
df["date"] = pd.to_numeric(df["date"], errors="coerce")

# Dopo la pulizia, rimuovi eventuali NaN e riconverti in int
df.dropna(subset=["minute", "date"], inplace=True)  # Elimina righe con NaN in questi campi
df["minute"] = df["minute"].astype(int)
df["date"] = df["date"].astype(int)

df['name'] = df['name'].str.replace('\xa0', ' ', regex=False)
df['tagline'] = df['tagline'].str.replace('\xa0', ' ', regex=False)
df['description'] = df['description'].str.replace('\xa0', ' ', regex=False)


In [5]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "movies_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Caricamento dati nel DB

In [6]:
import psycopg2
from sqlalchemy import create_engine, text

In [8]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "localhost"
DB_PORT = "5432"

try:
    conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
    conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
    cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database

    cur.execute(f"CREATE DATABASE {DB_NAME};")# Esegue il comando per creare il database con il nome specificato
    print(f"Database '{DB_NAME}' creato con successo.")
    
except psycopg2.errors.DuplicateDatabase:
    print(f"Il database '{DB_NAME}' esiste già.")

Il database 'ium_database' esiste già.


In [9]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [13]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS movies_data (
        id INT,
        name TEXT,
        date INT,
        tagline TEXT, 
        description TEXT,
        minute INT, 
        rating FLOAT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [14]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("movies_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

155